<a href="https://colab.research.google.com/github/aliash98/Search2Vec/blob/main/SkipGram_Validation_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [40]:
!pip install pandas
!pip install numpy

In [61]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import json
import warnings
import gc 
from tqdm import tqdm, trange
from scipy import spatial
import math

In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
df = pd.read_csv(r'./gdrive/MyDrive/Colab Notebooks/learned_weights.tsv', sep="\t")   
weights = df.values

# *** Note : this weights matrix doesn't contain the 0th redundant index

print(weights.shape)
print(weights[:5, :5])

(1544, 300)
[[-0.15826955  0.04051213 -0.08192925 -0.1244172  -0.06435126]
 [ 0.07748422  0.12065294 -0.06195808  0.07564399  0.03261204]
 [-0.02833261  0.10057643 -0.11756508 -0.0561972  -0.04967486]
 [ 0.1147184   0.04362245 -0.09093379 -0.07859519 -0.00983895]
 [-0.09325034  0.07769758 -0.15003568 -0.07568395 -0.04194207]]


In [44]:
vocab = json.load( open( r'./gdrive/MyDrive/Colab Notebooks/vocab.json' ) )
print(type(vocab))
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab[1])

<class 'dict'>
موبایل


## Finding the least cosine distance for a point

In [45]:
def least_cosine_distance(index, k):
  vocab_size, dim = weights.shape
  vector = weights[index, :]
  # print(vector.shape)
  # lets find the k nearest point to this
  distances = []
  indexes = []
  min_distances = []
  for i in range(0, vocab_size):
    distances.append(spatial.distance.cosine(vector, weights[i, :]))
  counter = 0
  while counter < k:
    temp_index = np.argmin(distances)
    if temp_index == index:
      distances[temp_index] = 1
      continue
    if not inverse_vocab[temp_index].endswith('cat'):
      distances[temp_index] = 1
      continue
    indexes.append(temp_index + 1)
    min_distances.append(distances[temp_index])
    distances[temp_index] = 1
    counter = counter + 1
  return indexes, min_distances 

    

In [46]:
vocab['هارد اکسترنال']

42

In [47]:
my_point = 42
indexes, min_distances = least_cosine_distance(my_point - 1, 10)
print(inverse_vocab[my_point])
print(min_distances)
print(indexes)
for index in indexes:
  print(inverse_vocab[index])
# a = weights[33, :]
# b = weights[714, :]
# print(1 - np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b)))

هارد اکسترنال
[0.0659602030782298, 0.0677344705554973, 0.08674923166200399, 0.08815149888200957, 0.08934095036303391, 0.09424868599398795, 0.09549136149136528, 0.10264476789405474, 0.10493785887553175, 0.12315909042587725]
[1113, 646, 580, 1049, 841, 660, 735, 658, 1456, 901]
هارد اکسترنال توشیباcat
لوازم جانبی هاردcat
حافظه اس اس دی (SSD)cat
هارد اکسترنال سیلیکون پاورcat
هارد اکسترنال سیگیتcat
سایر تجهیزات ذخیره‌سازی اطلاعاتcat
هارد اکسترنال اپیسرcat
سایر برندهای هارد اکسترنالcat
هارد اکسترنالcat
هارد اکسترنال ای دیتاcat


## Getting the validation set

In [48]:
validation_query = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_queries_validation.csv')
validation_clicks = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_clicks_validation.csv')
validation_products = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_products_validation.csv')

In [ ]:
# Rename
validation_clicks.rename(columns={'search_log_id':'_id'}, inplace=True)

In [50]:
print(validation_query.info())
print(validation_clicks.info())
print(validation_products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243638 entries, 0 to 243637
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   _id        243638 non-null  object
 1   raw_query  243638 non-null  object
 2   datetime   243638 non-null  object
dtypes: object(3)
memory usage: 5.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395070 entries, 0 to 395069
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   datetime         395070 non-null  object
 1   _id              395070 non-null  object
 2   rank             395070 non-null  int64 
 3   base_product_id  395070 non-null  object
dtypes: int64(1), object(3)
memory usage: 12.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45544 entries, 0 to 45543
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id 

## 10 nearaset category vocabulary

In [60]:
nearest_cat_vocab = {}
for word in vocab.keys():
  if not word.endswith('cat'):
    indexes, min_distances = least_cosine_distance(int(vocab[word]) - 1, 10)
    temp_list = []
    for index in indexes:
      temp_list.append(inverse_vocab[index])
    nearest_cat_vocab[word] = temp_list
  
print(nearest_cat_vocab)

{'<pad>': ['باتری گوشی و تبلتcat', 'دستگاه لاک خشک\u200cکنcat', 'ترمینال ریلی و شاخه\u200cایcat', 'مبلمان و جلو مبلیcat', 'گوشی الکاتلcat', 'تخمه و مغزهای طعم\u200cدارcat', 'لوازم حمامcat', 'لباس کار و یونیفرمcat', 'پمپ استخرcat', 'مداد چشم و ابروcat'], 'موبایل': ['گوشی ال\u200c جیcat', 'گوشی ایسوسcat', 'قلم نوری وکامcat', 'گوشی آنرcat', 'گلس تعمیراتی گوشی و تبلتcat', 'قاب، شاسی و درب گوشیcat', 'سایر برندهای تبلتcat', 'کارت خوان و صندوق فروشگاهیcat', 'سایر لوازم جانبی سیستم های صوتی و تصویریcat', 'گوشی بلک بریcat'], 'Ps4': ['خنک\u200c\u200cکننده و فنcat', 'سایر انواع فن و خنک\u200cکنندهcat', 'سایر برندهای کیبوردcat', 'منبع تغذیه UPScat', 'کنسول بازیcat', 'کابل و تبدیل HDMIcat', 'کنسول بازی نینتندوcat', 'باتری UPScat', 'سازهای زهیcat', 'باتری لپ\u200cتاپ سونیcat'], 'a52': ['باتری گوشی سامسونگcat', 'لپ\u200cتاپ سامسونگcat', 'فلش مموری اچ پیcat', 'فلش مموری کینگستونcat', 'محافظ و نگهدارنده کابلcat', 'قلم لمسیcat', 'باتری لپ\u200cتاپ ایسوسcat', 'گوشی الکاتلcat', 'فلش مموری سامسونگcat', 'فل

## Constructing the list of logs

In [64]:
logs = []
for i in range(1, 11):
  logs.append(1 / math.log(i + 1, 2))
print(logs)

[1.0, 0.6309297535714574, 0.5, 0.43067655807339306, 0.38685280723454163, 0.3562071871080222, 0.3333333333333333, 0.3154648767857287, 0.30102999566398114, 0.2890648263178878]


## Merging all datasets

In [52]:
validation_products.rename(columns={'_id':'base_product_id'}, inplace=True)
merged_click_product = pd.merge(validation_clicks, validation_products, how='inner', on=["base_product_id"])
print(merged_click_product.info())

merged_click_product = merged_click_product.drop_duplicates(subset=['_id'])
total_df = pd.merge(validation_query, merged_click_product, how='inner', on=["_id"])
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395070 entries, 0 to 395069
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   datetime         395070 non-null  object
 1   _id              395070 non-null  object
 2   rank             395070 non-null  int64 
 3   base_product_id  395070 non-null  object
 4   category_name    395070 non-null  object
 5   sellers          395070 non-null  object
dtypes: int64(1), object(5)
memory usage: 21.1+ MB
None


## Computing the ngdc

In [65]:
N, M = total_df.shape
print(N)

ndcg = 0

for i in range(N):
  temp_query = total_df.iloc[i, 1]
  temp_category = total_df.iloc[i,6] + 'cat'
  for i in range(10):
    if nearest_cat_vocab[temp_query][i] == temp_category:
      ndcg = ndcg + logs[i]
      break
    # print(i)
print(ndcg)





243638
112087.02059544601


In [69]:
print('Observed nDCG, according to the first clicks of each query, is ' + "{:.2f}".format(ndcg/N * 100) + '%')

Observed nDCG, according to the first clicks of each query, is 46.01%


In [ ]:
# query_cat_occur = merged_q_c.groupby(merged_q_c.columns.tolist(),as_index=False).size()


# merged_q_c['occur'] = merged_q_c.groupby('raw_query')['category_name'].transform('size')
# print(query_cat_occur.head(5))

    raw_query            category_name  size
0  12 pro max                شارژر اپل     2
1  12 pro max    محافظ صفحه گوشی آیفون     2
2  12 pro max   کیف و کاور گوشی و تبلت     9
3  12 pro max          گوشی اپل، آیفون   468
4         206  سایر لوازم تزئینی خودرو     1


In [ ]:
# print(query_cat_occur[query_cat_occur['raw_query'] == 'ماشین'])

     raw_query                  category_name  size
1942     ماشین                    اسکناس شمار     1
1943     ماشین         تجهیزات آشپزخانه صنعتی     7
1944     ماشین                دستگاه برش کاغذ     1
1945     ماشین                   دستگاه ماینر     1
1946     ماشین  دستگاه‌های بسته‌بندی و شیرینگ     2
1947     ماشین        سایر برندهای اسپرسو ساز     1
1948     ماشین               فایل‌های دانلودی     4
1949     ماشین               قطعات الکترونیکی     2
1950     ماشین          لوازم جانبی ابزارآلات     3
1951     ماشین        ماشین اصلاح و ریش‌ تراش   313
1952     ماشین                    ماشین شارژی     2
1953     ماشین                ماشین ظرف‌‌شویی     1
1954     ماشین   ماشین، قطار، کشتی اسباب بازی   729
